# Part B

In this part, We are going to follow same steps as part A but in this time, we are going to normalize datas.

But before we do that, we have to check whether  our gpu is run in our jupyter notebook.

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#this part is for gpu run in jupyter check. If you just use cpu based jupyter notebook, you can skip this.
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

True

If it returns true, we can assume that our jupyter is ready to use gpu option. (You can skip it if you just use gpu option)

## Check Data

In [3]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


We use the same table as we used in Part A. So we are going to follow the same steps as Part A did.

In [4]:
concrete_data.shape

(1030, 9)

Since it is a same table, we can see we have 1030 elements.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


We have to check whether there is an empty value in this table.

In [11]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

As you can see, there are no null values so we can use this datas.

Next, we have to split our datas into predictors and target.

Target will be 'Strength' and the others will be predictors.

In [12]:
concrete_data_columns = concrete_data.columns
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's check whether our predictors.

In [13]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


There is no 'Strength' section so we get a right one.

## Normalize predictors

Compared to Part A, Part B use normalized version of predictors.

This is done by subtract mean value and divide by standard deviation.

And we have to do this to all predictor elements.

In [16]:
predictors= (predictors - predictors.mean()) / predictors.std()
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Now, normalize is done, it is time to check target status.

In [17]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [18]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

Target also looks fine so we can move on to the next step.

## Import Keras and make regression model

As we talked before, it follows almost same steps as Part A. So it will be very similar to part A.

But it will give us a different outcome.

In [19]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


Using TensorFlow backend.


Since I use multiple gpu allow option, I have to put this 'sess' line to allow multiple gpu option jupyter notebook.

And from now on, we are going to build a regression_model.

In [20]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    #layer 10 , relu activation
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

After our model is ready, we have to split our data.

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

In [22]:
model = regression_model()

Now, It's time to train our model. Our epochs in this part is 50 times.

So we are going to train this for 50 times.

In [23]:
#fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

Epoch 1/50
721/721 [==============================] - 0s 497us/step - loss: 1571.4389
Epoch 2/50
721/721 [==============================] - 0s 42us/step - loss: 1550.9991
Epoch 3/50
721/721 [==============================] - 0s 40us/step - loss: 1530.2810
Epoch 4/50
721/721 [==============================] - 0s 39us/step - loss: 1509.1133
Epoch 5/50
721/721 [==============================] - 0s 40us/step - loss: 1487.1544
Epoch 6/50
721/721 [==============================] - 0s 39us/step - loss: 1464.4956
Epoch 7/50
721/721 [==============================] - 0s 41us/step - loss: 1441.3296
Epoch 8/50
721/721 [==============================] - 0s 43us/step - loss: 1416.8953
Epoch 9/50
721/721 [==============================] - 0s 40us/step - loss: 1391.6973
Epoch 10/50
721/721 [==============================] - 0s 40us/step - loss: 1365.7501
Epoch 11/50
721/721 [==============================] - 0s 40us/step - loss: 1339.0217
Epoch 12/50
721/721 [==============================] - 0s 39us

326.8294298486802

## MSE compared when we normalize our data

In [24]:
from sklearn.metrics import mean_squared_error

In [25]:
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

326.82943031279814 0.0


Finally, we are going to get a list of MSE so that we can get mean value and standard deviation.

In [26]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 145.65917487283355
MSE 2: 138.5803423637711
MSE 3: 83.43121614425314
MSE 4: 62.53047150547064
MSE 5: 55.29216595992301
MSE 6: 52.755264380988955
MSE 7: 53.82701135530441
MSE 8: 38.52106242503935
MSE 9: 40.24863181993799
MSE 10: 41.798181811582694
MSE 11: 38.12917385903763
MSE 12: 37.16566129023975
MSE 13: 44.03538466888724
MSE 14: 44.15001148853487
MSE 15: 38.146277035710106
MSE 16: 32.794969058731226
MSE 17: 35.75917564318018
MSE 18: 35.64967338784227
MSE 19: 35.54531909732757
MSE 20: 38.321390108768995
MSE 21: 33.53293920720665
MSE 22: 34.059093135846084
MSE 23: 31.335943339326235
MSE 24: 34.48919872790093
MSE 25: 35.657847043379995
MSE 26: 37.62956178535536
MSE 27: 31.676191262056914
MSE 28: 32.131370192592584
MSE 29: 38.09422350392758
MSE 30: 37.351726927031976
MSE 31: 33.36462001121546
MSE 32: 29.957634379562823
MSE 33: 30.096240528193107
MSE 34: 35.94297458438812
MSE 35: 35.00457588368635
MSE 36: 39.09190955054027
MSE 37: 32.01418311541906
MSE 38: 37.091827985152456
MSE 39

MSE in part A Mean: 54.85660058384991 

MSE in part B Mean: 42.8598274048752

In coclustion, MSE is getting much lower(almost 20%) when we normalize our datas.

This is due to normalize our datas(from the table).

Data normalization gets rid of a number of anomalies that can make analysis of the data more complicated. Some of those anomalies can crop up from deleting data, inserting more information, or updating existing information. So normalize datas are must go process to take when we build regression model.